 # **3. Dynamic pricing and personalised discounts**

* **Dynamically adjust prices** of each customer's new recommended product based on stock and sales trends
* Apply a **personalised discount** for each customer based on their value segment
* Adjust prices of products that **exceed highest** competitor benchmark or **fall below minimum** profit margin
  #####

In [1]:
# Import libraries
import pandas as pd

In [122]:
# Define method to join customer list with product list
def open_merge_files(customer_path, product_path):
    customer_list = pd.read_csv(customer_path)
    product_list = pd.read_csv(product_path)
    customer_price_list = pd.merge(customer_list, product_list,  how='left', left_on="Recommend1", right_on="product_id").drop(['Unnamed: 0_x', 'Unnamed: 0_y'], axis=1)
    customer_price_list = customer_price_list.drop(['Profile', 'Recency', 'Frequency', 'Monetary', 'RFM', 'Recommend2', 'Recommend3', 'product_id'], axis=1)
    #customer_price_list = customer_price_list.astype({'in_stock':'int', 'sales_old':'int', 'sales_change':'int', 'own_price':'int'})
    customer_price_list.loc[customer_price_list['med_price'].isna() & customer_price_list['min_price'].notna(), 'med_price'] = customer_price_list['min_price']
    customer_price_list.loc[customer_price_list['max_price'].isna() & customer_price_list['med_price'].notna(), 'max_price'] = customer_price_list['med_price']
    return customer_price_list


# Define method to dynamically adjust current prices based on stock levels and falling/rising sales figures
def apply_stock_demand_changes(df_list, lostock_hidemand=1.05, histock_hidemand=1.02, midstock_lodemand=0.98, lostock_lodemand=0.99, histock_lodemand=0.95):
    low_stock_value = 50 #Less than 50 units in stock
    mid_stock_value = 500 #Less than 500 units in stock
    high_stock_value = 800 #More than 800 units in stock
    low_demand_value = -40 #Sales fallen by at least 40%
    high_demand_value = 40 #Sales risen by at least 40%
    
    count_price_up = 0
    count_price_same = 0
    count_price_down = 0
   
    for index, row in df_list.iterrows():
        # Apply change for products with low stock and high demand
        if row['in_stock'] < low_stock_value and (row['sales_change'] / row['sales_old']) * 100 > high_demand_value:
            df_list.at[index, 'new_price'] = round(row['own_price'] * lostock_hidemand, 2)
            count_price_up+=1
        # Apply change for products with high stock and high demand
        elif row['in_stock'] > high_stock_value and (row['sales_change'] / row['sales_old']) * 100 > high_demand_value:
            df_list.at[index, 'new_price'] = round(row['own_price'] * histock_hidemand, 2)
            count_price_up+=1
        # Apply change for products with low stock and low demand
        elif row['in_stock'] < low_stock_value and (row['sales_change'] / row['sales_old']) * 100 < low_demand_value:
            df_list.at[index, 'new_price'] = round(row['own_price'] * lostock_lodemand, 2)
            count_price_down+=1
        # Apply change for products with mid stock and low demand
        elif row['in_stock'] < mid_stock_value and row['in_stock'] >= low_stock_value and (row['sales_change'] / row['sales_old']) * 100 < low_demand_value:
            df_list.at[index, 'new_price'] = round(row['own_price'] * midstock_lodemand, 2)
            count_price_down+=1
        # Apply change for products with high stock and low demand
        elif row['in_stock'] > high_stock_value and (row['sales_change'] / row['sales_old']) * 100 < low_demand_value:
            df_list.at[index, 'new_price'] = round(row['own_price'] * histock_lodemand, 2)
            count_price_down+=1
        else:
            # Otherwise maintain current price
            df_list.at[index, 'new_price'] = row['own_price']
            count_price_same+=1
    print('Dynamic pricing -- price increases:', str(count_price_up)+',', 'price decreases:', str(count_price_down)+',', 'no change:', count_price_same)
    return df_list   


# Define method to ensure prices are lower than our most expensive competitor
def apply_high_price_reduction(df_list, reduction_rate=0.85):
    count = 0
    for index, row in df_list.iterrows():
        if row['new_price'] > row['max_price'] and pd.notna(row['max_price']):
            df_list.at[index, 'new_price'] = round(row['max_price'] * reduction_rate, 2)
            count+=1
    print('Extreme high prices reduced:', count)
    return df_list


# Define method to apply discounts based of each customer's value/assigned segment
def apply_personalised_discount(df_list, high_discount=0.80, mid_discount=0.90, low_discount=0.95):
    for index, row in df_list.iterrows():
        if row['Segment'] == 'High value' or row['Segment'] == 'Mid value':
            df_list.at[index, 'discount_price'] = round(row['new_price']*mid_discount, 2)
        elif row['Segment'] == 'Win back':
            df_list.at[index, 'discount_price'] = round(row['new_price']*high_discount, 2)
        elif row['Segment'] == 'Improve' or row['Segment'] == 'Lost':
            df_list.at[index, 'discount_price'] = round(row['new_price']*low_discount, 2)
    return df_list


# Define method to ensure all prices above minimum profit margin, calculate new profit margin and calculate overall price change for customer
def check_not_below_minimum_price(df_list, column_name, profit_margin=1.1):
    count = 0
    for index, row in df_list.iterrows():
        if row[column_name] < (row['cost'] * profit_margin):
            df_list.at[index, column_name] = round(row['cost']* profit_margin, 2)
            count+=1
        new_profit = (df_list.at[index, column_name] - row['cost'])/row['cost'] * 100
        df_list.at[index, 'new_profit_%'] = round(new_profit, 1)
        total_discount = df_list.at[index, 'own_price'] - df_list.at[index, column_name] #
        df_list.at[index, 'total_price_reduction'] = total_discount
        discount_rate = df_list.at[index, 'total_price_reduction']/row['own_price'] * 100
        df_list.at[index, 'reduction_rate_%'] = round(discount_rate, 1)
    # Generate statistics on price changes
    discount_rate_highvalue = round(df_list[df_list['Segment'] == 'High value']['reduction_rate_%'].mean(), 1)
    discount_rate_midvalue = round(df_list[df_list['Segment'] == 'Mid value']['reduction_rate_%'].mean(), 1)
    discount_rate_improve = round(df_list[df_list['Segment'] == 'Improve']['reduction_rate_%'].mean(), 1)
    discount_rate_winback = round(df_list[df_list['Segment'] == 'Win back']['reduction_rate_%'].mean(), 1)
    discount_rate_lost = round(df_list[df_list['Segment'] == 'Lost']['reduction_rate_%'].mean(), 1)
    print('Prices below minimum adjusted:', count)
    print('Average price reduction:', str((round(df_list['reduction_rate_%'].mean(), 1)))+'%')
    print('Average price reduction per segment -- High value:', str(discount_rate_highvalue)+'%, ', 'Mid value:', str(discount_rate_midvalue)+'%, ', \
          'Improve:', str(discount_rate_improve)+'%, ', 'Win back:', str(discount_rate_winback)+'%, ', 'Lost:', str(discount_rate_lost)+'%, ')
    return df_list


In [123]:
# Open and merge customer and product lists
df_customer_price = open_merge_files("project_1.2_customers_recommended_products.csv", "project_1.3_data_products_prices_CLEANED.csv")

In [124]:
# Apply price changes, adjustments and discounts; 
df_customer_price = apply_stock_demand_changes(df_customer_price)  # Dynamic pricing
df_customer_price = apply_high_price_reduction(df_customer_price)  # Competitor benchmarking
df_customer_price = apply_personalised_discount(df_customer_price) # Personalised discount
df_customer_price = check_not_below_minimum_price(df_customer_price, 'discount_price') # Minimum profit margin // # Calculate discount + profit

Dynamic pricing -- price increases: 23, price decreases: 34, no change: 143
Extreme high prices reduced: 16
Prices below minimum adjusted: 20
Average price reduction: 13.0%
Average price reduction per segment -- High value: 13.5%,  Mid value: 12.6%,  Improve: 6.8%,  Win back: 19.0%,  Lost: 8.5%, 


In [125]:
# Display dataframe with price changes
display_new_prices = df_customer_price.copy()
display_new_prices = display_new_prices.drop(['description', 'med_price', 'sales_old', 'sales_new'], axis=1)
display_new_prices.head()

,Gender,Age,Latest purchase,Segment,Recommend1,name,own_price,cost,profit_rate_%,min_price,max_price,in_stock,sales_change,new_price,discount_price,new_profit_%,total_price_reduction,reduction_rate_%
0,Male,19,P000040,Mid value,P000039,Apple iPad Mini 64GB (2021),447.66,359.46,24.5,399.00,778.88,571,41,447.66,402.89,12.1,44.77,10.0
1,Male,21,P000349,Win back,P000344,ASUS Radeon RX 6650 XT Dual OC HDMI 3xDP 8G,279.99,163.19,71.6,234.99,421.70,961,63,285.59,228.47,40.0,51.52,18.4
2,Female,20,P000315,Improve,P000323,Roberts Radio Blutune 300,325.00,201.34,61.4,299.99,349.99,866,-5,308.75,293.31,45.7,31.69,9.8
3,Female,23,P000188,Win back,P000227,Zte MC801A,175.00,104.17,68.0,249.00,291.30,421,3,175.00,140.00,34.4,35.00,20.0
4,Female,31,P000088,Mid value,P000104,SteelSeries Aerox 9 W,146.60,82.64,77.4,99.99,159.99,212,-39,143.67,129.30,56.5,17.30,11.8


In [126]:
# Save file as .csv
df_customer_price.to_csv("project_1.3_dynamic_personalised_prices.csv")